In [123]:
from pymongo import MongoClient

# Construct the MongoDB connection string
username = 'backend_staging'
password = 'iQSo4fzAq4xtQHI2'
cluster_url = 'cluster0.tvyyu.mongodb.net'

# Note: Replace 'myDatabase' with your database name
connection_string = f"mongodb+srv://{username}:{password}@{cluster_url}/myDatabase?retryWrites=true&w=majority"

# Connect to the MongoDB cluster
client = MongoClient(connection_string)

# Now you can use 'client' to interact with your database
db = client.get_database("eweek-staging")  # Replace 'myDatabase' with your actual database name


In [71]:
pipeline = [
    {
        "$match": {
            "is_signin": False,
            "error_flag": {"$exists": False},
            "valid": True,
        }
    },
    {
        "$lookup": {
            "from": "org",
            "localField": "org",
            "foreignField": "_id",
            "as": "org_info"
        }
    },
    {
        "$unwind": "$org_info"
    },
    {
        "$lookup": {
            "from": "attendance_form",
            "localField": "eid",
            "foreignField": "eid",  # Assuming 'eid' is the common field in 'attendance_form'
            "as": "attendance_info"
        }
    },
    {
        "$unwind": {
            "path": "$attendance_info",
            "preserveNullAndEmptyArrays": True  # Optional: to keep documents even if there's no match in 'attendance_form'
        }
    },
    {
        "$group": {
            "_id": {
                "eid": "$eid",
                "org": "$org",
                "orgName": "$org_info.name"
            },
            "count": {"$sum": 1},
            "name": {"$first": "$attendance_info.name"}  # Assuming 'name' is the field you want from 'attendance_form'
        }
    },
    {
        "$sort": {
            "_id.org": 1,
            "count": -1
        }
    },
    {
        "$group": {
            "_id": "$_id.org",
            "eids": {
                "$push": {
                    "org": "$_id.org",
                    "orgName": "$_id.orgName",
                    "eid": "$_id.eid",
                    "count": "$count",
                    "name": "$name"  # Include 'name' in the output array
                }
            }
        }
    },
    # {
    #     "$project": {
    #         "eids": {"$slice": ["$eids", 25]}
    #     }
    # }
]


In [101]:
pipeline = [
    {
        "$match": {
            "is_signin": False,
            "valid": True
        }
    },
    {
        "$group": {
            "_id": "$eid",
            "count": {"$sum": 1}
        }
    },
    {
        "$lookup": {
            "from": "attendance_form",  # Assuming you want to get more details from the attendance_form again
            "localField": "_id",  # 'eid' is now the '_id' after the group stage
            "foreignField": "eid",
            "as": "attendance_details"
        }
    },
    {
        "$unwind": "$attendance_details"  # Unwind the attendance details to get the 'org' field
    },
    {
        "$lookup": {
            "from": "org",
            "localField": "attendance_details.org",
            "foreignField": "_id",
            "as": "org_info"
        }
    },
    {
        "$unwind": "$org_info"  # Unwind the organization details
    },
    {
        "$group": {
            "_id": "$org_info._id",
            "orgName": {"$first": "$org_info.name"},
            "eids": {
                "$push": {
                    "eid": "$_id",
                    "count": "$count",
                    "name": "$attendance_details.name"  # Assuming 'name' is a field in 'attendance_form'
                }
            }
        }
    }
]


In [107]:
results = db.attendance_form.aggregate(pipeline)
collapsed_data = {}

for result in results:
    org_name = result['orgName']
    eids_list = [{'eid': eid['eid'], 'name': eid['name'], 'count': eid['count']} for eid in result['eids']]
    collapsed_data[org_name] = eids_list


In [112]:
import pandas as pd

data = []
for org, eids in collapsed_data.items():
    for eid in eids:
        data.append([org, eid['eid'], eid['name'], eid['count']])
df = pd.DataFrame(data, columns=['org', 'eid', 'name', 'count'])

In [120]:
df_unique = df.drop_duplicates(subset=['eid'], keep='first')
# now just get first 15 rows for each org
# create separate csv for each org
for org in df_unique['org'].unique():
    df_org = df_unique[df_unique['org'] == org]
    df_org.to_csv(f'{org}.csv', index=False)
    print(f'Created {org}.csv')

Created AEI.csv
Created IEEE.csv
Created TxTPED.csv
Created EChO.csv
Created SASE.csv
Created BMESF.csv
Created SNAP.csv
Created Theta Tau.csv
Created WECE.csv
Created SWE.csv
Created OXE.csv
Created ASME.csv
Created TBP.csv
Created ESW.csv
Created ASCE.csv
Created AIChE.csv
Created SFE.csv


# Adding points

In [139]:
# Get org collection from db
org_collection = db.get_collection('org')
internal_collection = db.get_collection('internal_score')
external_collection = db.get_collection('external_score')

def add_points_internal_by_org_name(org_name, attendance_points=0, placement_points=0, hosting_points=0, other_points=0):
  """
  Add points to an organization's internal scores and update external scores using internal_score's _id.

  :param org_name: The name of the organization
  :param attendance_points: Integer, points for attendance
  :param placement_points: Integer, points for placement
  :param hosting_points: Integer, points for hosting
  :param other_points: Integer, points for other categories
  """
  # Find the organization by name
  org = org_collection.find_one({'name': org_name})
  if not org:
    print(f"Organization with name {org_name} not found.")
    return

  org_id = org['_id']

  # Update internal_score collection and retrieve the updated document's _id
  result = internal_collection.update_one(
    {'org': org_id},
    {'$inc': {
      'attendance_points': attendance_points,
      'placement_points': placement_points,
      'hosting_points': hosting_points,
      'other_points': other_points,
      'total_points': attendance_points + placement_points + hosting_points + other_points
    }},
    upsert=False
  )



In [186]:
data = """
SNAP	100
ASME	109
ASCE	101
TBP	101
OXE	100
BMESF	116
SWE	116
TxTPED	101
AIChE	101
WCAE	100
"""

lines = data.strip().split('\n')
for line in lines:
    org_name, points = line.split('\t')
    add_points_internal_by_org_name(org_name, other_points=int(points))
    # add_points_internal_by_org_name(org_name, attendance_points=int(points))

In [185]:
# Manual update
# add_points_internal_by_org_name(
  'EChO', 
  attendance_points=0, 
  placement_points=0, 
  hosting_points=0, 
  other_points=70
)

In [ ]:
# Updates external to be equivalent to internal

for internal_doc in internal_collection.find():
  internal_id = internal_doc['_id']
  attendance_points = internal_doc['attendance_points']
  placement_points = internal_doc['placement_points']
  hosting_points = internal_doc['hosting_points']
  other_points = internal_doc['other_points']
  total_points = internal_doc['total_points']

  external_collection.update_many(
    {'internal': internal_id},
    {'$set': {
      'attendance_points': attendance_points,
      'placement_points': placement_points,
      'hosting_points': hosting_points,
      'other_points': other_points,
      'total_points': total_points
    }}
  )